In [1]:
!pip install --upgrade pip
!pip install --upgrade setuptools
!pip install --upgrade "mxnet<2.0.0"
!pip install autogluon

  Using cached mxnet-1.9.1-py3-none-manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached graphviz-0.8.4-py2.py3-none-any.whl.metadata (6.4 kB)
Using cached mxnet-1.9.1-py3-none-manylinux2014_x86_64.whl (49.1 MB)
Using cached graphviz-0.8.4-py2.py3-none-any.whl (16 kB)
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
  Using cached autogluon-1.1.1-py3-none-any.whl.metadata (11 kB)
  Using cached autogluon.core-1.1.1-py3-none-any.whl.metadata (11 kB)
  Using cached autogluon.features-1.1.1-py3-none-any.whl.metadata (11 kB)
  Using cached autogluon.tabular-1.1.1-py3-none-any.whl.metadata (13 kB)
  Using cached autogluon.multimodal-1.1.1-py3-none-any.whl.metadata (12 kB)
  Using cached autogluon.timeseries-1.1.1-py3-none-any.whl.metadata (12 kB)
  Using cached scikit_learn-1.4.0-1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached bo

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor, FeatureMetadata

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [2]:
from sklearn.metrics import mean_squared_error
from autogluon.tabular import TabularDataset, TabularPredictor, FeatureMetadata

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [ ]:
#run from here
from google.colab import drive
drive.mount('/content/drive')

In [8]:
import pandas as pd
import numpy as np

# Total number of rows in the dataset
total_rows = 21000

# Sample data to represent your specified columns
data = {
    'udm.udm.infra.tcprtt._192.168.13.70_192.168.13.80.hist.bins._64_127.count': np.random.randint(0, 100, total_rows),
    'smf.smf.app.cadvisor.container_memory_rss': np.random.uniform(0, 16*1024**3, total_rows),  # up to 16 GB in bytes
    'ausf.ausf.app.cadvisor.container_memory_rss': np.random.uniform(0, 16*1024**3, total_rows),
    'amf.amf.app.cadvisor.container_memory_rss': np.random.uniform(0, 16*1024**3, total_rows),
    'upf.upf3.app.cadvisor.container_memory_rss': np.random.uniform(0, 16*1024**3, total_rows),
    'amf.amf.infra.tcpwin._192.168.13.80_192.168.13.82.snd_cwnd.hist.bins._4_7.count': np.random.randint(0, 100, total_rows),
    'udm.udm.infra.tcprtt._192.168.13.70_192.168.13.80.stat.avg': np.random.uniform(0, 500, total_rows),  # RTT in ms
    'udm.udm.app.cadvisor.container_memory_rss': np.random.uniform(0, 16*1024**3, total_rows),
    'udm.udm.infra.tcprtt._192.168.13.70_192.168.13.82.stat.avg': np.random.uniform(0, 500, total_rows),
    'udm.udm.app.cadvisor.container_memory_rss': np.random.uniform(0, 16*1024**3, total_rows),
    'upf.upf3.app.cadvisor.container_memory_rss': np.random.uniform(0, 16*1024**3, total_rows),
    'udm.udm.infra.tcprtt._192.168.13.70_192.168.13.82.stat.avg': np.random.randint(0, 100, total_rows)  # RTT in ms
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Define thresholds for each attribute based on their average range
thresholds = {
    'udm.udm.infra.tcprtt._192.168.13.70_192.168.13.80.hist.bins._64_127.count': 50,
    'smf.smf.app.cadvisor.container_memory_rss': 8*1024**3,  # Half of 16 GB in bytes
    'ausf.ausf.app.cadvisor.container_memory_rss': 8*1024**3,
    'amf.amf.app.cadvisor.container_memory_rss': 8*1024**3,
    'upf.upf3.app.cadvisor.container_memory_rss': 8*1024**3,
    'amf.amf.infra.tcpwin._192.168.13.80_192.168.13.82.snd_cwnd.hist.bins._4_7.count': 50,
    'udm.udm.infra.tcprtt._192.168.13.70_192.168.13.80.stat.avg': 250,  # Half of 500 ms
    'udm.udm.app.cadvisor.container_memory_rss': 8*1024**3,
    'udm.udm.infra.tcprtt._192.168.13.70_192.168.13.82.stat.avg': 50,
}

# Assign labels based on threshold crossing
conditions = []
for col, threshold in thresholds.items():
    conditions.append(df[col] > threshold)

# Count number of conditions met for each row
num_conditions_met = np.sum(conditions, axis=0)

# Assign label based on number of conditions met
df['label'] = (num_conditions_met >= 5).astype(int)

# Set 'amf.amf.app.five-g.RM.RegInitFail' based on num_conditions_met
'''
df['amf.amf.app.five-g.RM.RegInitFail'] = np.where(
    num_conditions_met < 5,
    0,
    10 - np.clip(num_conditions_met, 0, 10)
)
'''
# Save to CSV
df.to_csv('/generated_dataset.csv', index=False)
print("Dataset generated with labels and saved as 'generated_dataset_with_labels.csv'")


Dataset generated with labels and saved as 'generated_dataset_with_labels.csv'


In [ ]:
df=pd.read_csv('/generated_dataset.csv')


In [ ]:
print(len(df))
print(df.columns)

21000
Index(['udm.udm.infra.tcprtt._192.168.13.70_192.168.13.80.hist.bins._64_127.count',
       'smf.smf.app.cadvisor.container_memory_rss',
       'ausf.ausf.app.cadvisor.container_memory_rss',
       'amf.amf.app.cadvisor.container_memory_rss',
       'upf.upf3.app.cadvisor.container_memory_rss',
       'amf.amf.infra.tcpwin._192.168.13.80_192.168.13.82.snd_cwnd.hist.bins._4_7.count',
       'udm.udm.infra.tcprtt._192.168.13.70_192.168.13.80.stat.avg',
       'udm.udm.app.cadvisor.container_memory_rss',
       'udm.udm.infra.tcprtt._192.168.13.70_192.168.13.82.stat.avg', 'label',
       'amf.amf.app.five-g.RM.RegInitFail'],
      dtype='object')


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Load the generated dataset
data_raw_tab = pd.read_csv('/generated_dataset.csv')
num_cycles = 300
rows_per_cycle = 70
# Identify cycles with failures
cycleLable_train_arr = []
for i in range(num_cycles):
    dt1_train = data_raw_tab['label'][rows_per_cycle*i:rows_per_cycle*(i+1)]
    if dt1_train.iloc[0] != 0:
        cycleLable_train_arr.append(i)

# Downsample and create final training dataset
ind_train_60 = []
data_train_1to12_12_tab = pd.DataFrame()

for i_cycle in range(num_cycles):
    if i_cycle in cycleLable_train_arr:
        tmp = data_raw_tab.iloc[i_cycle*rows_per_cycle+10:i_cycle*rows_per_cycle+16, 2:-2]  # Skip timestamp and label
        data_train_1to12_12_tab = pd.concat([data_train_1to12_12_tab, tmp])
        for i in range(6):
            ind_train_60.append(i_cycle*rows_per_cycle+59)
    else:
        for offset in [0, 10, 20, 30, 40, 50]:
            tmp = data_raw_tab.iloc[i_cycle*rows_per_cycle+offset:i_cycle*rows_per_cycle+offset+1, 2:-2]
            data_train_1to12_12_tab = pd.concat([data_train_1to12_12_tab, tmp])
        for i in range(6):
            ind_train_60.append(i_cycle*rows_per_cycle+59)

data_failure_60 = data_raw_tab.iloc[ind_train_60, :]['amf.amf.app.five-g.RM.RegInitFail']
data_train_12_mean_failure_60_tab = data_train_1to12_12_tab
data_train_12_mean_failure_60_tab['fail60'] = data_failure_60.to_numpy()
data_train_12_mean_failure_60_noTimeLable_tab = data_train_12_mean_failure_60_tab.reset_index(drop=True)

# Save the final training dataset to CSV
data_train_12_mean_failure_60_noTimeLable_tab.to_csv('/final_training_dataset.csv', index=False)
print("Final training dataset generated and saved as 'final_training_dataset.csv'")

KeyError: 'amf.amf.app.five-g.RM.RegInitFail'

In [13]:
%%time
dir_default = "drive/My Drive/model"
#dir_default = f"{dir_base_name}_1025_train_12_failure_60_reg_cpu_4h"
data_input_tab = pd.read_csv('/content/final_training_dataset.csv')

predictor = TabularPredictor(
    label='fail60',
    problem_type='regression',
    path=dir_default,
    verbosity=2,
).fit(
    train_data=data_input_tab,
    presets="good_quality",
    ag_args_fit={"num_gpus": 0},
     time_limit=14400
)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       9.88 GB / 12.67 GB (77.9%)
Disk Space Avail:   74.25 GB / 107.72 GB (68.9%)
Presets specified: ['good_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input da

CPU times: user 22.8 s, sys: 515 ms, total: 23.3 s
Wall time: 39.3 s


In [ ]:
data_train_table = pd.read_csv('drive/My Drive/ML5G-PS-005_train.csv') #raw data
data_test_table = pd.read_csv('drive/My Drive/ML5G-PS-005_test.csv') #raw data

In [ ]:
data_train_failure = []
data_train_normal = []
for i in range(0,600):
  dt1_train = data_train_table['label'][70*i:70*(i+1)]
  if dt1_train[70*i] != 'normal':
    data_train_failure.append(i)
  else:
    data_train_normal.append(i)

In [ ]:
#train data + test data
fulldata = pd.concat([data_train_table, data_test_table])

In [ ]:

dt = fulldata.copy() #LSTMで用いるデータ
#delete time, label columns
del dt['time']
del dt['label']

In [ ]:
dt_udm = dt.iloc[:, 2391:2477]

In [ ]:

feature_importance = ['amf.amf.app.cadvisor.container_memory_rss',
'ausf.ausf.app.cadvisor.container_memory_rss',
'igw.igw1.app.cadvisor.container_memory_rss',
'igw.igw2.app.cadvisor.container_memory_rss',
'nrf.nrf.app.cadvisor.container_memory_rss',
'smf.smf.app.cadvisor.container_memory_rss',
'udm.udm.app.cadvisor.container_memory_rss',
'upf.upf1.app.cadvisor.container_memory_rss',
'upf.upf2.app.cadvisor.container_memory_rss',
'upf.upf3.app.cadvisor.container_memory_rss']

In [ ]:


dt_importantcolumn = pd.DataFrame()
dt_importantcolumn = pd.concat([dt_importantcolumn, dt_udm])

In [ ]:

for i_column in feature_importance:
  dt_importantcolumn[i_column] = dt[i_column][:]

In [ ]:
dt_importantcolumn.insert(0, 'amf.amf.app.five-g.RM.RegInitFail', dt['amf.amf.app.five-g.RM.RegInitFail'][:])



In [ ]:
dt_importantcolumn.to_csv('drive/My Drive/dt_importantcolumn_feature_udm_rss_goodquality_1114.csv')


In [ ]:
#read
dt_importantcolumn = pd.read_csv('drive/My Drive/dt_importantcolumn_feature_udm_rss_goodquality_1114.csv')


In [ ]:


del dt_importantcolumn['Unnamed: 0']

In [ ]:


dt_train_importantcolumn = dt_importantcolumn[:len(data_train_table)][:]
dt_test_importantcolumn = dt_importantcolumn[len(data_train_table):][:]

In [ ]:


data_train_normal_375 = data_train_normal[:375]
data_train_normal_75 = data_train_normal[375:]
data_train_failure_125 = data_train_failure[:125]
data_train_failure_25 = data_train_failure[125:]

In [ ]:


#separate train data(600 cycles) into 500 cycles and 100 cycles
data_train_500_table = dt_train_importantcolumn
data_train_100_table = pd.DataFrame()

In [ ]:
for i_cycle in range(600):
  if i_cycle in data_train_normal_75 or i_cycle in data_train_failure_25:
    data_train_100_table = pd.concat([data_train_100_table, dt_train_importantcolumn.iloc[i_cycle*70:i_cycle*70+70, :]])
    data_train_500_table = data_train_500_table.drop(range(i_cycle*70, i_cycle*70+70))

In [ ]:
data_train_500_table.to_csv('drive/My Drive/data_train_500_table_1114.csv')
data_train_100_table.to_csv('drive/My Drive/data_train_100_table_1114.csv')

In [ ]:
data_train_500_table = pd.read_csv('drive/My Drive/data_train_500_table_1114.csv')
data_train_100_table = pd.read_csv('drive/My Drive/data_train_100_table_1114.csv')

In [ ]:


del data_train_500_table['Unnamed: 0']
del data_train_100_table['Unnamed: 0']

In [ ]:

data_train_500_all = data_train_normal_375 + data_train_failure_125
data_train_500_all.sort()

In [ ]:
#training data
ind_train_60 = []
data_train_1to60_12_tab = pd.DataFrame()

for i_cycle in data_train_500_all:
  if i_cycle in data_train_failure_125:
    tmp = dt_train_importantcolumn.iloc[i_cycle*70+10:i_cycle*70+22, :]
    data_train_1to60_12_tab = pd.concat([data_train_1to60_12_tab, tmp])
    for i in range(12):
      ind_train_60.append(i_cycle*70+59)

  else:
    tmp = dt_train_importantcolumn.iloc[i_cycle*70:i_cycle*70+1, :]
    data_train_1to60_12_tab = pd.concat([data_train_1to60_12_tab, tmp])

    tmp = dt_train_importantcolumn.iloc[i_cycle*70+5:i_cycle*70+6, :]
    data_train_1to60_12_tab = pd.concat([data_train_1to60_12_tab, tmp])

    tmp = dt_train_importantcolumn.iloc[i_cycle*70+10:i_cycle*70+11, :]
    data_train_1to60_12_tab = pd.concat([data_train_1to60_12_tab, tmp])

    tmp = dt_train_importantcolumn.iloc[i_cycle*70+15:i_cycle*70+16, :]
    data_train_1to60_12_tab = pd.concat([data_train_1to60_12_tab, tmp])

    tmp = dt_train_importantcolumn.iloc[i_cycle*70+20:i_cycle*70+21, :]
    data_train_1to60_12_tab = pd.concat([data_train_1to60_12_tab, tmp])

    tmp = dt_train_importantcolumn.iloc[i_cycle*70+25:i_cycle*70+26, :]
    data_train_1to60_12_tab = pd.concat([data_train_1to60_12_tab, tmp])

    tmp = dt_train_importantcolumn.iloc[i_cycle*70+30:i_cycle*70+31, :]
    data_train_1to60_12_tab = pd.concat([data_train_1to60_12_tab, tmp])

    tmp = dt_train_importantcolumn.iloc[i_cycle*70+35:i_cycle*70+36, :]
    data_train_1to60_12_tab = pd.concat([data_train_1to60_12_tab, tmp])

    tmp = dt_train_importantcolumn.iloc[i_cycle*70+40:i_cycle*70+41, :]
    data_train_1to60_12_tab = pd.concat([data_train_1to60_12_tab, tmp])

    tmp = dt_train_importantcolumn.iloc[i_cycle*70+45:i_cycle*70+46, :]
    data_train_1to60_12_tab = pd.concat([data_train_1to60_12_tab, tmp])

    tmp = dt_train_importantcolumn.iloc[i_cycle*70+50:i_cycle*70+51, :]
    data_train_1to60_12_tab = pd.concat([data_train_1to60_12_tab, tmp])

    tmp = dt_train_importantcolumn.iloc[i_cycle*70+55:i_cycle*70+56, :]
    data_train_1to60_12_tab = pd.concat([data_train_1to60_12_tab, tmp])

    for i in range(12):
      ind_train_60.append(i_cycle*70+59)

data_failure_60 = dt_train_importantcolumn.iloc[ind_train_60, :]['amf.amf.app.five-g.RM.RegInitFail']
data_train_12_mean_failure_60_tab=data_train_1to60_12_tab
data_train_12_mean_failure_60_tab['fail60']=data_failure_60.to_numpy()
data_train_12_mean_failure_60_noTimeLable_tab=data_train_12_mean_failure_60_tab.reset_index()

print(data_failure_60.shape)
data_train_12_mean_failure_60_noTimeLable_tab.head(6)


In [ ]:
del data_train_12_mean_failure_60_noTimeLable_tab['index'] #index

In [ ]:
data_train_12_mean_failure_60_noTimeLable_tab.to_csv('drive/My Drive/data_train_12_mean_fai

In [ ]:
data_train_12_mean_failure_60_noTimeLable_tab = pd.read_csv('drive/My Drive/data_train_12_mean_failure_60_noTimeLable_tab_1116.csv')


In [ ]:
del data_train_12_mean_failure_60_noTimeLable_tab['Unnamed: 0']

In [ ]:

data_train_12_mean_failure_60_noTimeLable_tab

In [ ]:
%%time
dir_default = "drive/My Drive/model"
#dir_default = f"{dir_base_name}_1121_train_normal_cycle_96columns_good_noindex_reg_cpu_4h"
data_input_tab = data_train_12_mean_failure_60_noTimeLable_tab
predictor = TabularPredictor(
    label='fail60', # ターゲット
    # label='amf.amf.app.five-g.RM.RegInitFail', # ターゲット
    problem_type='regression',
    path=dir_default, # モデルを保存するディレクトリ名
    verbosity=2,
    # eval_metric="f1" ## 評価指標 Defaults to ‘accuracy’ for binary and multiclass classification and ‘root_mean_squared_error’ for regression
#     eval_metric="roc_auc" # 評価指標
#     eval_metric="precision" # 評価指標
).fit(
    train_data=data_input_tab, # トレーニングデータ
    # presets="good_quality", ## default = [‘medium_quality_faster_train’]
    presets="good_quality",
#     presets="optimize_for_deployment",
#     presets=["good_quality","optimize_for_deployment"],
    # excluded_model_types = ['CAT'],
#     feature_metadata=feature_metadata,
    # ag_args_ensemble={"fold_fitting_strategy": "sequential_local"},
#    ag_args_fit={"num_gpus": 0},  # GPU の使用
     time_limit=14400 # [seconds]
#    time_limit=3600, # [seconds]
#     hyperparameters=hyperparameters
)

In [ ]:
predictor = TabularPredictor.load("drive/My Drive/model/autogluon_1121_train_normal_cycle_96columns_good_noindex_reg_cpu_4h",require_version_match=False)


In [12]:
test = pd.read_csv('/generated_dataset.csv')
score = predictor.predict(test.iloc[0:1])
score

0    2.021022
Name: fail60, dtype: float32

In [ ]:
predictor.leaderboard(
    data_train_12_mean_failure_60_noTimeLable_tab,
    silent=True
)